In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv
/kaggle/input/bert/keras/bert_base_en_uncased/2/config.json
/kaggle/input/bert/keras/bert_base_en_uncased/2/tokenizer.json
/kaggle/input/bert/keras/bert_base_en_uncased/2/metadata.json
/kaggle/input/bert/keras/bert_base_en_uncased/2/model.weights.h5
/kaggle/input/bert/keras/bert_base_en_uncased/2/assets/tokenizer/vocabulary.txt


 <h1> Spam Classification using BERT</h1>

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import keras_nlp

import warnings
warnings.filterwarnings('ignore')

2024-07-25 04:09:04.892464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 04:09:04.892603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 04:09:05.019883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Import the dataset

In [3]:
import pandas as pd

df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='ISO-8859-1')
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df1 = df.drop(['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 3'], axis='columns')
df1.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df2 = df1.rename(columns={'v1': 'Category', 'v2': 'Message'})
df2.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df2.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [7]:
df2['spam']=df2['Category'].apply(lambda x: 1 if x=='spam' else 0)
df2.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


## Split the Data into Training and Test Sets

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df2['Message'],df2['spam'], stratify=df2['spam'])

In [9]:
X_train.head(4)

4414    Yeah I should be able to, I'll text you when I...
3323    I don wake since. I checked that stuff and saw...
2021    I don't have anybody's number, I still haven't...
5092    My fri ah... Okie lor,goin 4 my drivin den go ...
Name: Message, dtype: object

# Import the BERT model and obtain embedding vectors

### Now let's import the BERT model and get embedding vectors for a few sample statements


In [10]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [11]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.5132726 , -0.88845724, ..., -0.7474888 ,
        -0.75314724,  0.91964495],
       [-0.8720837 , -0.50543994, -0.94446695, ..., -0.8584751 ,
        -0.7174536 ,  0.88083   ]], dtype=float32)>

## Embedding vectors

### Get embedding vectors for a few sample words and compare them using cosine similarity

In [12]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911089]], dtype=float32)

### Values near to 1 indicate high similarity, while values near to 0 indicate significant differences. For example, comparing the words "banana" and "grapes" might yield a similarity score of 0.99, as they are both fruits and thus share similar contextual meanings.

In [14]:
cosine_similarity([e[0]],[e[3]])

array([[0.8470389]], dtype=float32)

### Comparing "banana" with "Jeff Bezos" might yield a similarity score of 0.84, which is still relatively high but not as close as the 0.99 similarity score obtained with "grapes". This reflects that while "banana" and "Jeff Bezos" have some contextual overlap, "banana" and "grapes" are much more closely related in terms of their contextual meanings.


In [15]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872035]], dtype=float32)

"Jeff Bezos" and "Elon Musk" are more similar to each other than "Jeff Bezos" and "banana", as indicated by their higher cosine similarity score. This suggests that "Jeff Bezos" and "Elon Musk" share more contextual similarities, likely due to their comparable roles in technology and business, compared to the more disparate context between "Jeff Bezos" and "banana".

# Model Building

There are two types of models you can build in TensorFlow:

1. **Sequential**
2. **Functional**

So far, we have built a Sequential model. Below, we will build a Functional model. More information on these two types of models can be found [here](https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057).


In [16]:
# Define input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

# Define BERT preprocessor and encoder from keras_nlp
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased", trainable=True)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")

# Preprocess text input
encoder_inputs = preprocessor(text_input)

# Get BERT outputs
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]

# Neural network layers
dropout = tf.keras.layers.Dropout(0.1, name="dropout")(pooled_output)
dense = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout)

# Construct the final model
model = tf.keras.Model(inputs=text_input, outputs=dense)

# Compile the model (optional, depending on your use case)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'preprocessor.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)   │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_preprocessor   │ [(None, 512),     │          0 │ text[0][0]        │
│ (BertPreprocessor)  │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 768),     │ 109,482,2… │ bert_preprocesso… │
│ (BertBackbone)      │ (None, 512, 768)] │            │ bert_preprocesso… │
│                     │                   │            │ bert_preprocesso… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ bert_backbone[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        769 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
len(X_train)

4179

In [18]:
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')
# ]

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=METRICS)

## Model Training

In [19]:
# model.fit(X_train, y_train, epochs=5)

In [20]:
# model.evaluate(X_test, y_test)

# Inference

In [21]:
# reviews = [
#     'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
#     'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
#     'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
#     'Hey Sam, Are you coming for a cricket game tomorrow',
#     "Why don't you wait 'til at least wednesday to see if you get your ."
# ]
# model.predict(reviews)